# SQLChatMessageHistory - 데이터베이스 기반 대화 기록 관리

이 노트북에서는 SQLAlchemy를 사용하여 대화 기록을 영구적으로 저장하고 관리하는 방법을 학습합니다.

## 주요 학습 내용
- SQLChatMessageHistory를 사용한 데이터베이스 기반 대화 저장
- 다중 사용자 및 세션 관리
- RunnableWithMessageHistory와의 통합
- ConfigurableFieldSpec을 통한 동적 설정

## SQL과 SQLAlchemy 소개

### SQL (Structured Query Language)
[SQL](https://en.wikipedia.org/wiki/SQL)은 관계형 데이터베이스 관리 시스템(RDBMS)에서 데이터를 관리하기 위해 설계된 도메인 특화 언어입니다. 특히 엔티티와 변수 간의 관계를 포함하는 구조화된 데이터를 다루는 데 유용합니다.

### SQLAlchemy
[SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy)는 Python용 오픈 소스 SQL 툴킷이자 객체 관계 매퍼(ORM)입니다. MIT 라이선스로 배포되며, Python 애플리케이션에서 데이터베이스를 쉽게 다룰 수 있게 해줍니다.

### SQLChatMessageHistory
LangChain의 `SQLChatMessageHistory` 클래스는 SQLAlchemy가 지원하는 모든 데이터베이스에 채팅 기록을 저장할 수 있습니다. SQLite, PostgreSQL, MySQL 등 다양한 데이터베이스를 지원합니다.

## 환경 설정

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()

True

## SQLChatMessageHistory 기본 사용법

SQLChatMessageHistory를 사용하려면 다음 2가지를 제공해야 합니다:

1. **session_id**: 세션의 고유 식별자 (사용자 이름, 이메일, 채팅 ID 등)
2. **connection**: 데이터베이스 연결 문자열 (SQLAlchemy의 create_engine 함수에 전달됨)

In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
# SQLite를 사용하여 로컬 파일로 저장
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", 
    connection="sqlite:///sqlite.db"  # SQLite 데이터베이스 파일 경로
)

## 메시지 추가 및 조회

In [4]:
# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 홍기야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!"
)

# AI 메시지를 추가합니다.
chat_message_history.add_ai_message(
    "안녕 홍기, 만나서 반가워. 나도 잘 부탁해!"
)

In [5]:
# 저장된 채팅 메시지 기록을 확인
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내 이름은 홍기야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 홍기, 만나서 반가워. 나도 잘 부탁해!', additional_kwargs={}, response_metadata={})]

## LangChain과 통합하여 사용하기

SQLChatMessageHistory를 RunnableWithMessageHistory와 함께 사용하여 자동으로 대화 기록을 관리할 수 있습니다.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [7]:
# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지
        ("system", "You are a helpful assistant."),
        # 대화 기록을 위한 Placeholder
        MessagesPlaceholder(variable_name="chat_history"),
        # 사용자 질문
        ("human", "{question}"),
    ]
)

# LLM 체인 생성
chain = prompt | ChatOpenAI(model_name="gpt-4o") | StrOutputParser()

## 다중 사용자 및 세션 관리

실제 애플리케이션에서는 여러 사용자와 각 사용자의 여러 대화를 관리해야 합니다.

In [8]:
# 사용자별, 대화별로 SQLChatMessageHistory를 가져오는 함수
def get_chat_history(user_id, conversation_id):
    """
    사용자 ID와 대화 ID를 기반으로 SQLChatMessageHistory 인스턴스를 반환합니다.
    
    Args:
        user_id: 사용자 고유 식별자
        conversation_id: 대화 고유 식별자
    
    Returns:
        SQLChatMessageHistory: 해당 사용자와 대화의 메시지 기록
    """
    return SQLChatMessageHistory(
        table_name=user_id,  # 사용자별로 다른 테이블 사용
        session_id=conversation_id,  # 대화별로 다른 세션 ID
        connection="sqlite:///sqlite.db",
    )

## ConfigurableFieldSpec 설정

동적으로 사용자와 대화를 전환할 수 있도록 설정 필드를 정의합니다.

In [9]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

# 설정 가능한 필드 정의
config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [10]:
# 메시지 기록 기능이 포함된 체인 생성
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수
    input_messages_key="question",  # 입력 메시지의 키
    history_messages_key="chat_history",  # 대화 기록 메시지의 키
    history_factory_config=config_fields,  # 설정 가능한 필드
)

## 실제 사용 예제

In [11]:
# 사용자1의 대화1 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [16]:
# 첫 번째 메시지
chain_with_history.invoke(
    {"question": "안녕 반가워, 내 이름은 홍기야"}, 
    config
)

'안녕하세요, 홍기님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [17]:
# 후속 질문 - 이전 대화 내용을 기억함
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고?"}, 
    config
)

'당신의 이름은 홍기입니다. 맞나요?'

## 다른 대화로 전환

In [18]:
# 같은 사용자의 다른 대화로 전환
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 새로운 대화에서는 이전 대화의 내용을 모름
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고?"}, 
    config
)

'죄송하지만, 아직 당신의 이름을 알려주지 않으셨습니다. 이름을 알려주시면 기억하겠습니다.'

## 장점과 활용 방안

### 장점
1. **영구 저장**: 메모리와 달리 서버가 재시작되어도 대화 기록이 유지됩니다
2. **확장성**: 다양한 데이터베이스 지원 (SQLite, PostgreSQL, MySQL 등)
3. **다중 사용자**: 여러 사용자와 세션을 효율적으로 관리
4. **유연성**: 테이블명과 세션 ID를 통한 세밀한 제어

### 활용 방안
- **챗봇 서비스**: 사용자별 대화 기록 관리
- **고객 상담**: 상담 이력 추적 및 분석
- **교육 플랫폼**: 학습자별 진도 및 대화 관리
- **협업 도구**: 팀별, 프로젝트별 대화 분리

### 주의사항
- SQLite는 개발 및 테스트용으로 적합하며, 프로덕션에서는 PostgreSQL이나 MySQL 같은 데이터베이스 사용 권장
- 대용량 트래픽 처리 시 데이터베이스 연결 풀링 고려
- 개인정보 보호를 위한 데이터 암호화 및 접근 제어 구현 필요